<a href="https://colab.research.google.com/github/aharrisonau/Task3.2Dr2/blob/main/Task_3_2D_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SIT796 Reinforcement Learning
##Distinction Task 3.2: Exact policy iteration implementation for MDPs

This task follows up 1.2C, which was the definition of a truck trailer reversing problem.

In this task, the aim is to set up a policy value mapping for truck reversal.  To achieve this, the state space has been mapped to 4 variables;
- pivot X [0, 2, 4,... 30] metres
- pivot Y [-10, -8, -6, .... 35] metres
- prime mover angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians
- trailer angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians

This gives 16 x 21 x 10 x 10 = 33600 individual co-ordinates, which is a balance between space state size and resolution

Reward of 1 is achieved with;
- pivot X,Y = (10,0)
- trailer angle = 0
- pm angle = [16pi/10, 18pi/10, 0, 2pi/10, 4pi/10]  NOTE: other PM angles represent a jack-knifed trailer and should not occur


In [49]:
# preliminaries
import numpy as np
import pandas as pd
import math
pi = math.pi
import cmath


In [50]:
# the calculations will be done in numpy for speed
# first define the State Space co-ordinated for the array
SSpivotX = np.linspace(0,30,16)
SSpivotY = np.linspace(-10,30,21)
SSpmAngle = np.linspace(0,2*pi,11)
SStrlAngle = np.linspace(0,2*pi,11)

# then set the reward space and terminal nodes as all zeros
#Rewards = np.zeros((len(SSpivotX),len(SSpivotY),len(SSpmAngle),len(SStrlAngle)))
#TerminalStates = np.zeros((len(SSpivotX),len(SSpivotY),len(SSpmAngle),len(SStrlAngle)))

# and lastly set the terminals for backed in (no next state)
#TerminalStates[(5,5,range(-2,3),0)] = 1
#Rewards[(5,5,range(-2,3),0)] = 1

In [51]:
# define a function that duplicates the step function of the gym environment

def nextStateFN(state, action): # function is a modified version of the environment step function
    import cmath
    import math
    pi = math.pi
    moveBasis = 2.0  # meters the Prime mover moves for action of +1

    pivX,pivY,pmAng,trlAng = state
    move,steer = action
    TruckDefinition = np.array( [5.0, 2.5, 10.0, 2.5])
    """
    in each step, 
    - the prime mover will go back moveBasis (at the pivot)
    - the prime mover will rotate (front relative to pivot)
    - the trailer will rotate due to its relative angle
    """

    
    # move the pivot point
    pmUnitVector = np.array([cmath.rect(1,pmAng).real,cmath.rect(1,pmAng).imag])
    [pivX,pivY] = [pivX,pivY] + move * moveBasis * pmUnitVector
    
    # rotate the trailer (--and adjust its back location-- not needed)
    pm_trlAngle= pmAng - trlAng #relative trailer angle.
    trlAngleDelta=math.asin(math.sin(pm_trlAngle)*move*0.1/TruckDefinition[2]) # the change in trailer angle  
    trlAng = trlAng + trlAngleDelta
    trlAng = (trlAng + 2*pi)%(2*pi) # keep the angle between 0 and 2*pi
    
    # rotate the prime mover (--and adjust the front location-- not needed)
    pmAngleDelta=math.asin(math.tan(pi/4*steer)*move*moveBasis/TruckDefinition[0]) # the change in PM angle due to wheel steering
    pmAng = pmAng + pmAngleDelta
    pmAng = (pmAng + 2*pi)%(2*pi)   


    nextState = [pivX,pivY,pmAng,trlAng]

    if abs(pivX - TruckDefinition[2]) <= 0.5 and \
              abs(pivY - 0) <= 0.5 and \
              (trlAng+0.174)%(2*pi) <= 2*0.174 : # trailer within 10deg of straight
        reward = 1.0
    else:
        reward = 0.0
    
    done = reward == 1.0

    return nextState,reward

Now we step through every state in the state space and use the nextState function to get the following state for a given action.

The actions that we will use are;
move = constant -1 (which is moving 2m back)
steer = [-1, -0.2, 0, 0.2, 1] (corresponding from full lock right to full lock left)

We will check if the next state is a terminal state (out of bounds) or has a reward as well and record those results

These can then be used for value iteration for the episodic event chain

In [52]:
# set up some bins to pull state calcuations into the nearest state space points
binsPivotX = SSpivotX+1
binsPivotY = SSpivotY+1
binsPmAngle = SSpmAngle+pi/10
binsTrlAngle = SStrlAngle+pi/10

In [53]:
# set the allowable actions
move = -1.0 #only going backwards
steering = np.array([-1.0, -0.2, 0, 0.2, 1.0])

In [54]:
# use a nested dictionary structure to store the results
# levels will be
# 1 - PivotX bins
# 2 - PivotY bins
# 3 - PmAngle bins
# 4 - TrlAngle bins
# 5 - action index

nextStateDict = {}

for pivX in SSpivotX:
  print(pivX,",",end=" ") # heartbeat to follow progress
  nextStateDict[pivX] = {}
  for pivY in SSpivotY:
    nextStateDict[pivX][pivY] = {}
    for pmAngle in SSpmAngle:
      nextStateDict[pivX][pivY][pmAngle] = {}
      for trlAngle in SStrlAngle:
        nextStateDict[pivX][pivY][pmAngle][trlAngle] = {}
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=0
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=0
        # note that the value of a state is stored against the state
        for steer in steering:
          # this is now one of the 5 actions applied at a particular state
          nextStateDict[pivX][pivY][pmAngle][trlAngle][steer] = {}
          state = [pivX,pivY,pmAngle,trlAngle]
          nextState, reward = nextStateFN(state,(-1.0,steer))
          if state == nextState:
            print("Same:",state)
          next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
          
          # check if the state has gone out of bounds.  If so, zero it out
          if  not SSpivotX[0] <= next_pivX <= SSpivotX[-1]:
            nextState = 0
          elif not SSpivotY[0] <= next_pivY <= SSpivotY[-1]:
            nextState = 0
          else:
            # we need to round the result to the nearest state space
            next_pivX = SSpivotX[np.searchsorted(binsPivotX,next_pivX)]
            next_pivY = SSpivotY[np.searchsorted(binsPivotY,next_pivY)]
            next_pmAngle = SSpmAngle[np.searchsorted(binsPmAngle,next_pmAngle)]
            next_trlAngle = SStrlAngle[np.searchsorted(binsTrlAngle,next_trlAngle)]
            nextState = [next_pivX,next_pivY,next_pmAngle,next_trlAngle]

          if nextState == 0:
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = 0
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = 0
          else:
            # assign that reward for that action at that state
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = reward
            # if the reward is greater than the current value of the state, update it
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"] = max(reward, 
                        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"])   
            # record what the next state for that action on this state is                                             
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = nextState
            

0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 

Now we can use a state value iteration process, as well as a policy iteration process to determine a map of optimum values and actions.

Essentially, we pass through the whole state space, checking all the actions at each point and assessing the value at the next space that action reaches.

We record the best action to take, plus copy a slightly discounted value from that next state to the current state.

This pass is repeated until sufficient convergence is achieved.

In [58]:
# loop through the state space and update the rewards
stillConverging = True
count = 0
gamma = 0.99  # discount factor
changeLimit = 0.001

while stillConverging:
  
  change = 0.0

  for pivX in SSpivotX:
    print(pivX,",",end=" ") # heartbeat to follow progress
    for pivY in SSpivotY:
      for pmAngle in SSpmAngle:
        for trlAngle in SStrlAngle:
          # reset variables to track the best value and action for this state
          bestValue = 0
          bestAction = 0
          # loop through the actions
          for steer in steering:
            nextState = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"]
            if nextState != 0: # if it is zero, that action was out-of-bounds
              next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
              nextStateValue = nextStateDict[next_pivX][next_pivY][next_pmAngle][next_trlAngle]["value"]
              if nextStateValue > bestValue: # if this action was better
                bestValue = nextStateValue   # update the best value
                bestAction = steer        # update the best action
          
          oldValue = nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]
          increase = max(bestValue*gamma - oldValue, 0.0)
          change = max(increase,change) # check if this value increase is the biggest
          if increase > 0:
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=bestValue*gamma
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=bestAction

  print(count,change)
  count += 1 # update the loop count
  stillConverging = change > changeLimit # stop the loop once the largest change is small

0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 0 0.99
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 1 0.9801
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 2 0.9702989999999999
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 3 0.96059601
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 4 0.9509900498999999
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 5 0.9414801494009999
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 6 0.9320653479069899
0.0 , 2.0 , 4.0 , 6.0 , 8.0 , 10.0 , 12.0 , 14.0 , 16.0 , 18.0 , 20.0 , 22.0 , 24.0 , 26.0 , 28.0 , 30.0 , 7 0.92274469442792
0.0 , 2.0 , 

Once the value and best action map is complete, it can be examined.  As it is four dimensional, it is best viewed as a map of the x-y co-ords for a fixed prime mover and trailer angle set.  In this case, the map is for angles of 4*pi/10

In [60]:
map = pd.DataFrame(index=SSpivotY)

for pivX in SSpivotX:
  col = []
  for pivY in SSpivotY:
    pmAngle = SSpmAngle[0]
    trlAngle = SStrlAngle[0]
    col.append(nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"])
    #print(pivX,pivY,pmAngle,trlAngle,":",nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]) 

  map[pivX] = col
  #print(pivX,col)

pd.options.display.float_format = '{:.2f}'.format
map


,0.00,2.00,4.00,6.00,8.00,10.00,12.00,14.00,16.00,18.00,20.00,22.00,24.00,26.00,28.00,30.00
-10.00,0,0.00,0.00,0.93,0.94,0.94,0.94,0.93,0.92,0.94,0.94,0.94,0.94,0.93,0.92,0.91
-8.00,0,0.00,0.00,0.93,0.94,0.95,0.94,0.93,0.92,0.95,0.95,0.95,0.94,0.93,0.92,0.91
-6.00,0,0.00,0.00,0.93,0.92,0.91,0.92,0.93,0.92,0.96,0.96,0.95,0.94,0.93,0.92,0.91
-4.00,0,0.00,0.00,0.88,0.89,0.91,0.90,0.91,0.92,0.97,0.96,0.95,0.94,0.93,0.92,0.91
-2.00,0,0.00,0.00,0.88,0.89,0.90,0.90,0.90,0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91
0.00,0,0.00,0.00,0.88,0.89,0.90,1.00,0.99,0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91
2.00,0,0.00,0.00,0.88,0.89,0.90,0.90,0.90,0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91
4.00,0,0.00,0.00,0.88,0.89,0.91,0.90,0.91,0.92,0.97,0.96,0.95,0.94,0.93,0.92,0.91
6.00,0,0.00,0.00,0.93,0.92,0.91,0.92,0.93,0.92,0.96,0.96,0.95,0.94,0.93,0.92,0.91
8.00,0,0.00,0.00,0.93,0.94,0.95,0.94,0.93,0.92,0.95,0.95,0.95,0.94,0.93,0.92,0.91


From any state point that has a value, this value has been propogated back through a chain of actions that leads to the goal.  This can be followed forwards as a the optimum path to achieve the goal

In [79]:
startPointIndex = [15,13,0,0]
np.set_printoptions(precision=2)

currentState = np.array([SSpivotX[startPointIndex[0]],\
                SSpivotY[startPointIndex[1]],\
                SSpmAngle[startPointIndex[2]],\
                SStrlAngle[startPointIndex[3]]])
pivX,pivY,pmAngle,trlAngle = currentState               
notThereYet = True

while notThereYet:
  print("State:", currentState, end=" ")
  pivX,pivY,pmAngle,trlAngle = currentState
  value = nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]
  bestAction = nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]
  print("Value:",f"{value:.2f}","Best Action:",bestAction, end=" ")
  nextState = np.array(nextStateDict[pivX][pivY][pmAngle][trlAngle][bestAction]["nextSS"])
  print("Next state:",nextState)
  if value >= 1:
    notThereYet = False
  currentState = nextState

State: [30. 16.  0.  0.] Value: 0.91 Best Action: -1.0 Next state: [28.   16.    0.63  0.  ]
State: [28.   16.    0.63  0.  ] Value: 0.92 Best Action: -0.2 Next state: [26.   14.    0.63  6.28]
State: [26.   14.    0.63  6.28] Value: 0.93 Best Action: -0.2 Next state: [24.   12.    0.63  6.28]
State: [24.   12.    0.63  6.28] Value: 0.94 Best Action: -0.2 Next state: [22.   10.    0.63  6.28]
State: [22.   10.    0.63  6.28] Value: 0.95 Best Action: -0.2 Next state: [20.    8.    0.63  6.28]
State: [20.    8.    0.63  6.28] Value: 0.96 Best Action: -0.2 Next state: [18.    6.    0.63  6.28]
State: [18.    6.    0.63  6.28] Value: 0.97 Best Action: -0.2 Next state: [16.    4.    0.63  6.28]
State: [16.    4.    0.63  6.28] Value: 0.98 Best Action: -0.2 Next state: [14.    2.    0.63  6.28]
State: [14.    2.    0.63  6.28] Value: 0.99 Best Action: 1.0 Next state: [12.    0.    0.    6.28]
State: [12.    0.    0.    6.28] Value: 1.00 Best Action: 0 Next state: [10.  0.  0.  0.]


Code below is for exporting PDF version of ipynb file

In [80]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Task 3.2D.ipynb')

--2021-04-12 13:27:02--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2021-04-12 13:27:03 (31.2 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

Mounted at /content/drive/




Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/Task 3.2D.ipynb to pdf
[NbConvertApp] Writing 71837 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bi

'File Download Unsuccessful. Saved in Google Drive'